In [50]:
import numpy as np
import opensimplex
import random
from time import sleep
import scipy.linalg as sla
import numba 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray

In [57]:
createPopCode = open("cudaKernels/createInitialPopulation.cu").read()
createRndmCode = open("cudaKernels/createRandomState.cu").read()

mod_rndm = SourceModule(createRndmCode % {"SEED" : 0}, no_extern_c = True)
mod_pop = SourceModule(createPopCode, no_extern_c = True)

createPopulation = mod_pop.get_function("create_initial_population")
createRandom = mod_rndm.get_function("initkernel")

actv_blcks = 4
rndm_st_sz = 48
thrds_per_blk = 1024
population_size = 200
time_frame = 90
nmbr_trks = 10
trk_cp = 20
number_of_stores =5
number_of_products = 12
probability_of_no_truck = 0.25

rndm_nmb_gntr = cuda.mem_alloc(rndm_st_sz * actv_blcks * thrds_per_blk)
population = cuda.mem_alloc(population_size * time_frame * nmbr_trks * (trk_cp+1) * 4)
population_cpu = np.zeros((population_size , time_frame , nmbr_trks , (trk_cp+1)), dtype =np.int32,order ='C')
cuda.memcpy_htod(population, population_cpu)

createRandom(rndm_nmb_gntr, block=(32,8,4), grid=(4,1,1))
createPopulation(population, np.array(population_size, dtype=np.int32) , np.array(time_frame, dtype=np.int32) ,
                 np.array(nmbr_trks, dtype=np.int32) , np.array(trk_cp, dtype=np.int32), 
                 np.array(number_of_stores, dtype=np.int32), np.array(number_of_products, dtype=np.int32), 
                 np.array(probability_of_no_truck, dtype=np.float32), rndm_nmb_gntr, block=(1024,1, 1), grid=(4,1,1))
cuda.memcpy_dtoh(population_cpu, population)

In [59]:
np.sum(population_cpu[:,:,:,0] == -1) / (1800)


25.147777777777776